mtask处理代码
1. 得到最基础的mtask_relative，会有识别不出来的情况
通过运行改版后的train.py，并且要把lerobot_dataset里面
```
        # 训练的时候要用
        # if self.language_tip_mode:
        #     tip_filename = f"mtask_{self.language_tip_mode}.jsonl"
        #     tip_file = self.root / "meta" / tip_filename
        #     if not tip_file.exists():
        #         raise FileNotFoundError(f"训练需要的文件不存在: {tip_file}")
        #     self.modified_tasks = load_modified_tasks(tip_file)
        #     print("self.modified_tasks举例：",self.modified_tasks[1][0])
```
和factory里面
```
        # exclude_features += ["observation.images.side_depth", "observation.images.side_depth_is_pad"]
```


2. 然后补完原始的数值

In [7]:
parent_file="/home/zhonglinyue/code/lerobot_diy/training_dataset/0902/stack/meta/"
pure_file=parent_file+"modified_tasks_pure.jsonl"
modified_file=parent_file+"full_modified_tasks.jsonl"
relative_file=parent_file+"mtask_relative.jsonl"
grid_5cm=parent_file+"mtask_5cm.jsonl"
grid_2cm=parent_file+"grid_2cm.jsonl"
objest=["accessory","router"]
removr_same=parent_file+"filtered_full_modified_tasks.jsonl"


In [5]:
import json
import re
from pathlib import Path


def parse_coords(text, key):
    pattern = rf"{key} at \((-?\d+\.?\d*), (-?\d+\.?\d*), (-?\d+\.?\d*)\)"
    match = re.search(pattern, text)
    if match:
        return tuple(map(float, match.groups()))
    return None


def replace_coords(text, key, coords):
    pattern = rf"{key} at \([^)]*\)"
    if coords is None:
        return text
    repl = f"{key} at ({coords[0]:.3f}, {coords[1]:.3f}, {coords[2]:.3f})"
    if re.search(pattern, text):
        return re.sub(pattern, repl, text)
    else:
        if "|" in text:
            return text.strip() + " | " + repl
        else:
            return text.strip() + " | " + repl


def linear_interpolate(start, end, n):
    """n = gap size (包含两端，所以要生成 n-1 个点)"""
    if start is None or end is None:
        return [None] * (n - 1)
    result = []
    for i in range(1, n):
        frac = i / n
        interp = tuple(s + (e - s) * frac for s, e in zip(start, end))
        result.append(interp)
    return result


def process_file(input_file, output_file, objects):
    lines = [json.loads(line) for line in Path(input_file).read_text().splitlines()]
    modified = 0

    episodes = {}
    for entry in lines:
        episodes.setdefault(entry["episode_index"], []).append(entry)

    for ep, entries in episodes.items():
        entries.sort(key=lambda x: x["frame_index"])

        for obj in objects:
            # 提取每一帧该 object 的坐标
            coords_list = [parse_coords(e["task"], obj) for e in entries]

            # ---- 补全开头缺失 ----
            first_valid = next((c for c in coords_list if c is not None), None)
            if first_valid:
                for i in range(len(coords_list)):
                    if coords_list[i] is None:
                        coords_list[i] = first_valid
                        modified += 1
                    else:
                        break

            # ---- 补全结尾缺失 ----
            last_valid = next((c for c in reversed(coords_list) if c is not None), None)
            if last_valid:
                for i in range(len(coords_list) - 1, -1, -1):
                    if coords_list[i] is None:
                        coords_list[i] = last_valid
                        modified += 1
                    else:
                        break

            # ---- 插值中间缺失 ----
            i = 0
            while i < len(coords_list):
                if coords_list[i] is None:
                    j = i
                    while j < len(coords_list) and coords_list[j] is None:
                        j += 1
                    # i-1 和 j 是两端有值的点
                    if i > 0 and j < len(coords_list):
                        interp_vals = linear_interpolate(coords_list[i - 1], coords_list[j], j - i + 1)
                        for k in range(i, j):
                            coords_list[k] = interp_vals[k - i]
                            modified += 1
                    i = j
                else:
                    i += 1

            # 更新 task 文本
            for idx, e in enumerate(entries):
                e["task"] = replace_coords(e["task"], obj, coords_list[idx])

    with open(output_file, "w") as f:
        for ep in sorted(episodes):
            for entry in sorted(episodes[ep], key=lambda x: x["frame_index"]):
                f.write(json.dumps(entry, ensure_ascii=False) + "\n")

    print(f"修改成功: {modified}")


# if __name__ == "__main__":
#     objects = ["gripper", "accessory", "router"]  # ✅ 这里定义需要补的物体
#     process_file("pure.jsonl", "modified.jsonl", objects)


if __name__ == "__main__":
    objects = ["gripper", "accessory", "router"] 
    # 修改这里即可
    process_file(pure_file, modified_file, objects=objects)


修改成功: 16609


In [8]:
import json
import re
from pathlib import Path

def parse_coords(text, key):
    pattern = rf"{key} at \((-?\d+\.?\d*), (-?\d+\.?\d*), (-?\d+\.?\d*)\)"
    match = re.search(pattern, text)
    if match:
        return tuple(map(float, match.groups()))
    return None

def replace_coords(text, key, coords):
    pattern = rf"{key} at \([^)]*\)"
    if coords is None:
        return text
    repl = f"{key} at ({coords[0]:.3f}, {coords[1]:.3f}, {coords[2]:.3f})"
    if re.search(pattern, text):
        return re.sub(pattern, repl, text)
    else:
        if "|" in text:
            return text.strip() + " | " + repl
        else:
            return text.strip() + " | " + repl

def average_coords(coords_list):
    """输入 [(x,y,z), ...] 返回平均坐标"""
    n = len(coords_list)
    summed = [0.0, 0.0, 0.0]
    for c in coords_list:
        for i in range(3):
            summed[i] += c[i]
    return tuple(s / n for s in summed)

def remove_duplicate_frames(input_file, output_file, objects=["gripper","accessory","router"]):
    lines = [json.loads(line) for line in Path(input_file).read_text().splitlines()]

    # 按 (episode_index, frame_index) 分组
    frames = {}
    for e in lines:
        key = (e["episode_index"], e["frame_index"])
        frames.setdefault(key, []).append(e)

    result = []
    duplicates_removed = 0

    for key, entries in frames.items():
        # 如果多条 task 完全相同，保留一条
        task_texts = [e["task"] for e in entries]
        unique_task_texts = set(task_texts)
        if len(unique_task_texts) == 1:
            # 全部一样，保留一条
            result.append(entries[0])
            duplicates_removed += len(entries) - 1
        else:
            # 不同，求每个 object 坐标平均
            avg_task = entries[0]["task"]  # 基础文本
            for obj in objects:
                coords_obj_list = [parse_coords(e["task"], obj) for e in entries]
                coords_obj_list = [c for c in coords_obj_list if c is not None]
                if coords_obj_list:
                    avg_coord = average_coords(coords_obj_list)
                    avg_task = replace_coords(avg_task, obj, avg_coord)
            # 构建新 entry
            new_entry = {
                "episode_index": entries[0]["episode_index"],
                "frame_index": entries[0]["frame_index"],
                "task": avg_task
            }
            result.append(new_entry)
            duplicates_removed += len(entries) - 1

    # 按 episode_index 和 frame_index 排序
    result.sort(key=lambda x: (x["episode_index"], x["frame_index"]))

    # 写回文件
    with open(output_file, "w") as f:
        for e in result:
            f.write(json.dumps(e, ensure_ascii=False) + "\n")

    print(f"总行数: {len(lines)}, 去重后行数: {len(result)}, 删除重复或合并行: {duplicates_removed}")


总行数: 23150, 去重后行数: 15355, 删除重复行: 7795
⚠️ 有 7760 条同一 (episode, frame) 但 task 不同的记录，已写入 diff_tasks.jsonl


In [3]:
# 检查
import json
import re
from pathlib import Path

def parse_coords(text, key):
    pattern = rf"{key} at \((-?\d+\.?\d*), (-?\d+\.?\d*), (-?\d+\.?\d*)\)"
    return re.search(pattern, text) is not None

def check_file(input_file, objects):
    lines = [json.loads(line) for line in Path(input_file).read_text().splitlines()]
    missing = []

    for entry in lines:
        task = entry["task"]
        frame_missing = [obj for obj in objects if not parse_coords(task, obj)]
        if frame_missing:
            missing.append({
                "episode_index": entry["episode_index"],
                "frame_index": entry["frame_index"],
                "missing": frame_missing,
                "task": task
            })

    if missing:
        print(f"⚠️ 有 {len(missing)} 个 task 缺少物体坐标：")
        for m in missing[:20]:  # 只打印前20个，避免太长
            print(f"  ep{m['episode_index']} frame{m['frame_index']} 缺失: {m['missing']}")
    else:
        print("✅ 所有 task 都包含了完整的物体坐标！")

if __name__ == "__main__":
    objects = ["gripper", "accessory", "router"]  # 需要检查的物体
    check_file(modified_file, objects)


✅ 所有 task 都包含了完整的物体坐标！


In [ ]:
# 取代哦重复的
import json
import re
from pathlib import Path

objects = ["gripper", "accessory", "router"]

def parse_coords(text, key):
    pattern = rf"{key} at \((-?\d+\.?\d*), (-?\d+\.?\d*), (-?\d+\.?\d*)\)"
    match = re.search(pattern, text)
    if match:
        return tuple(map(float, match.groups()))
    return None

def replace_coords(text, key, coords):
    pattern = rf"{key} at \([^)]*\)"
    if coords is None:
        return text
    repl = f"{key} at ({coords[0]:.3f}, {coords[1]:.3f}, {coords[2]:.3f})"
    if re.search(pattern, text):
        return re.sub(pattern, repl, text)
    else:
        if "|" in text:
            return text.strip() + " | " + repl
        else:
            return text.strip() + " | " + repl

def remove_duplicate_frames(input_file, output_file):
    lines = [json.loads(line) for line in Path(input_file).read_text().splitlines()]

    # 按 (episode_index, frame_index) 分组
    frames = {}
    for e in lines:
        key = (e["episode_index"], e["frame_index"])
        frames.setdefault(key, []).append(e)

    result = []
    duplicates_removed = 0

    for key, entries in frames.items():
        # 检查 task 是否完全一致
        task_set = set(e["task"] for e in entries)
        if len(task_set) == 1:
            # 全部相同，保留一条
            result.append(entries[0])
            duplicates_removed += len(entries) - 1
        else:
            # task 不同，求每个 object 平均位置
            avg_coords = {}
            for obj in objects:
                coords_list = [parse_coords(e["task"], obj) for e in entries if parse_coords(e["task"], obj) is not None]
                if coords_list:
                    n = len(coords_list)
                    avg_coords[obj] = tuple(sum(c[i] for c in coords_list)/n for i in range(3))
                else:
                    avg_coords[obj] = None

            # 构建新的 task
            new_task = entries[0]["task"]
            for obj in objects:
                new_task = replace_coords(new_task, obj, avg_coords[obj])

            # 保存一条新数据
            new_entry = entries[0].copy()
            new_entry["task"] = new_task
            result.append(new_entry)
            duplicates_removed += len(entries) - 1

    # 按 episode_index 和 frame_index 排序
    result.sort(key=lambda x: (x["episode_index"], x["frame_index"]))

    # 写回文件
    with open(output_file, "w") as f:
        for e in result:
            f.write(json.dumps(e, ensure_ascii=False) + "\n")

    print(f"总行数: {len(lines)}, 去重后行数: {len(result)}, 删除或合并重复行: {duplicates_removed}")


if __name__ == "__main__":
    remove_duplicate_frames(modified_file, removr_same)


变成mtask_relative

In [14]:
import json
import re
from pathlib import Path

def parse_coords(text, key):
    pattern = rf"{key} at \((-?\d+\.?\d*), (-?\d+\.?\d*), (-?\d+\.?\d*)\)"
    match = re.search(pattern, text)
    if match:
        return tuple(map(float, match.groups()))
    return None

def clean_task(text):
    """
    去掉原 task 中的 '| gripper at ... | accessory at ... | router at ...' 坐标信息
    """
    return re.sub(r"\|.*?at \((-?\d+\.?\d*), (-?\d+\.?\d*), (-?\d+\.?\d*)\)", "", text).strip()

def process_file(input_file, output_file, objects):
    lines = [json.loads(line) for line in Path(input_file).read_text().splitlines()]

    with open(output_file, "w") as f:
        for entry in lines:
            original_task = clean_task(entry["task"])
            g = parse_coords(entry["task"], "gripper")

            rel_parts = []
            if g:
                for obj in objects:
                    o = parse_coords(entry["task"], obj)
                    if o:
                        rel = tuple(o_i - g_i for o_i, g_i in zip(o, g))
                        rel_parts.append(
                            f"{obj} position relative to gripper is ({rel[0]:.3f}, {rel[1]:.3f}, {rel[2]:.3f})"
                        )

            # 拼接最终 task
            if rel_parts:
                entry["task"] = original_task + ", " + ", ".join(rel_parts)
            else:
                entry["task"] = original_task

            f.write(json.dumps(entry, ensure_ascii=False) + "\n")

if __name__ == "__main__":
    objects = ["accessory", "router"]
    process_file(removr_same, relative_file, objects)


5cm格子

In [17]:
import json
import re
from pathlib import Path

def parse_coords(text, key):
    pattern = rf"{key} at \((-?\d+\.?\d*), (-?\d+\.?\d*), (-?\d+\.?\d*)\)"
    match = re.search(pattern, text)
    if match:
        return tuple(map(float, match.groups()))
    return None

def clean_task(text):
    """
    去掉原 task 中的 '| gripper at ... | accessory at ... | router at ...' 坐标信息
    """
    return re.sub(r"\|.*?at \((-?\d+\.?\d*), (-?\d+\.?\d*), (-?\d+\.?\d*)\)", "", text).strip()

def process_file(input_file, output_file, objects):
    lines = [json.loads(line) for line in Path(input_file).read_text().splitlines()]

    with open(output_file, "w") as f:
        for entry in lines:
            original_task = clean_task(entry["task"])
            g = parse_coords(entry["task"], "gripper")

            rel_parts = []
            if g:
                for obj in objects:
                    o = parse_coords(entry["task"], obj)
                    if o:
                        # 计算相对 gripper 的坐标
                        rel = tuple(o_i - g_i for o_i, g_i in zip(o, g))
                        # 转换到 5cm 格子并取整
                        grid = tuple(round(val / 0.05) for val in rel)
                        # 格式化为两位整数
                        grid_str = tuple(f"{num:02d}" if num >= 0 else f"-{abs(num):02d}" for num in grid)
                        rel_parts.append(
                            f"{obj} position relative to gripper is ({grid_str[0]}, {grid_str[1]}, {grid_str[2]}) in 5cm grid units"
                        )

            # 拼接最终 task
            if rel_parts:
                entry["task"] = original_task + ", " + ", ".join(rel_parts)
            else:
                entry["task"] = original_task

            f.write(json.dumps(entry, ensure_ascii=False) + "\n")

if __name__ == "__main__":
    objects = ["accessory", "router"]  # 想要计算的物体
    process_file(removr_same, grid_5cm, objects)


# 2cm

In [18]:
import json
import re
from pathlib import Path

def parse_coords(text, key):
    pattern = rf"{key} at \((-?\d+\.?\d*), (-?\d+\.?\d*), (-?\d+\.?\d*)\)"
    match = re.search(pattern, text)
    if match:
        return tuple(map(float, match.groups()))
    return None

def clean_task(text):
    """
    去掉原 task 中的 '| gripper at ... | accessory at ... | router at ...' 坐标信息
    """
    return re.sub(r"\|.*?at \((-?\d+\.?\d*), (-?\d+\.?\d*), (-?\d+\.?\d*)\)", "", text).strip()

def process_file_2cm(input_file, output_file, objects):
    lines = [json.loads(line) for line in Path(input_file).read_text().splitlines()]

    with open(output_file, "w") as f:
        for entry in lines:
            original_task = clean_task(entry["task"])
            g = parse_coords(entry["task"], "gripper")

            rel_parts = []
            if g:
                for obj in objects:
                    o = parse_coords(entry["task"], obj)
                    if o:
                        # 计算相对 gripper 的坐标
                        rel = tuple(o_i - g_i for o_i, g_i in zip(o, g))
                        # 转换到 2cm 格子并取整
                        grid = tuple(round(val / 0.02) for val in rel)
                        # 格式化为两位整数
                        grid_str = tuple(f"{num:02d}" if num >= 0 else f"-{abs(num):02d}" for num in grid)
                        rel_parts.append(
                            f"{obj} position relative to gripper is ({grid_str[0]}, {grid_str[1]}, {grid_str[2]}) in 2cm grid units"
                        )

            # 拼接最终 task
            if rel_parts:
                entry["task"] = original_task + ", " + ", ".join(rel_parts)
            else:
                entry["task"] = original_task

            f.write(json.dumps(entry, ensure_ascii=False) + "\n")

if __name__ == "__main__":
    objects = ["accessory", "router"]  # 想要计算的物体
    process_file_2cm(removr_same, grid_2cm, objects)
